###다항회귀와 다중회귀

* 다항회귀
 > 하나의 특성값을 이용하여 차수를 높이는 회귀분석
* 다중회귀
  > 2개 이상의 특성값을 이용하는 회귀분석

In [ ]:
from sklearn.preprocessing import (다항특성 클래스)

poly = PolynomialFeatures(include_bias=False) # degree : 기본값 2, include_bias(절편값 : 1)
#poly.fit([[2,3]])
#print(poly.transform([[2,3]]))
print(poly.fit_transform([[2,3]]))
poly.get_feature_names_out()

In [ ]:
import pandas as pd

csv_data = pd.read_csv("./Data/perch.csv")
perch_length = csv_data[["length"]]
perch_weight = csv_data["weight"]

In [ ]:
print(poly.fit_transform(perch_length))

In [ ]:
import pandas as pd

csv_data = pd.read_csv("./Data/perch_full.csv")
perch_full = csv_data[["length","height","width"]]
perch_weight = csv_data["weight"]

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)
print(train_data.shape, test_data.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(train_data, train_target)
print("훈련데이터의 정확도 : ", lr_model.score(train_data, train_target))
print("테스트데이터의 정확도 : ", lr_model.score(test_data, test_target))

In [ ]:
train_poly = poly.fit_transform(train_data)
print(train_poly.shape)
poly.get_feature_names_out()

In [ ]:
test_poly = poly.fit_transform(test_data)

lr_poly_model = LinearRegression()
lr_poly_model.fit(train_poly, train_target)
print("훈련데이터의 정확도 : ", lr_poly_model.score(train_poly, train_target))
print("테스트데이터의 정확도 : ", lr_poly_model.score(test_poly, test_target))

In [ ]:
poly2 = PolynomialFeatures(degree=5, include_bias=False)
train_poly2 = poly2.fit_transform(train_data)
print(train_poly2.shape)

In [ ]:
test_poly2 = poly2.transform(test_data)

lr_poly_model.fit(train_poly2, train_target)
print("훈련데이터의 정확도 : ", lr_poly_model.score(train_poly2, train_target))
print("테스트데이터의 정확도 : ", lr_poly_model.score(test_poly2, test_target))

###과대적합(Overfitting) & 과소적합(Underfitting)

![](https://images.velog.io/images/arittung/post/d68462e5-77cb-47fc-acee-a8cb5ea1925d/image.png)

과대 적합(Overfitting)

* 모델이 훈련 세트에서는 좋은 성능을 내지만 검증 세트에서는 낮은 성능을 내는 경우.

* 훈련 세트와 검증 세트에서 측정한 성능의 간격이 큼. (== 분산이 큼(high variance))

* 과대 적합의 주요 원인
: 훈련 세트에 충분히 다양한 샘플이 포함되지 않음.

* 해결 방법 :

> 1) 훈련 세트에 충분히 다양한 샘플을 포함시킴.

> 2) 훈련 샘플을 더 모을 수 없는 경우, 모델이 훈련 세트에 집착하지 않도록 가중치를 제한함. (= 모델의 복잡도를 낮춤)

> 3) 훈련 데이터의 잡음을 줄임.(Outlier, Error 제거)

과소 적합(Underfitting)

* 훈련 세트와 검증 세트의 성능에는 차이가 크지 않지만 모두 낮은 성능을 내는 경우. 훈련 세트와 검증 세트의 성능이 서로 가까워지면 성능 자체가 낮음.

* 과소적합된 모델을 '편향이 크다(high bias)'라고도 함.

* 해결 방법 :

> 1) 복잡도가 더 높은 모델(파라미터가 더 많은 모델) 사용

> 2) 가중치 규제 완화

### 규제
* 릿지(Ridge) = (L2규제) = $(가중치)^2$
* 라쏘(Lasso) = (L1규제) = |가중치|

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
train_scaled = ss.fit_transform(train_poly2)
test_scaled = ss.transform(test_poly2)

In [ ]:
lr_poly_model.fit(train_scaled, train_target)
print("훈련데이터의 정확도 : ", lr_poly_model.score(train_scaled, train_target))
print("검증데이터의 정확도 : ", lr_poly_model.score(test_scaled, test_target))

In [ ]:
from sklearn.linear_model import (릿지 규제 클래스)

ridge = Ridge() #alpha(규제강도):하이퍼파라미터(사용자 지정 값), 알파값이 커지면 규제강도 커짐
ridge.fit(train_scaled, train_target)
print("훈련데이터의 정확도 : ", ridge.score(train_scaled, train_target))
print("검증데이터의 정확도 : ", ridge.score(test_scaled, test_target))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

train_score = []
test_score = []

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
  ridge = Ridge(alpha=alpha)
  ridge.fit(train_scaled, train_target)
  train_score.append(ridge.score(train_scaled, train_target))
  test_score.append(ridge.score(test_scaled, test_target))

plt.plot(np.log10(alpha_list), train_score, color="red")
plt.plot(np.log10(alpha_list), test_score, color="blue")
plt.xlabel("alpha")
plt.ylabel("R^2")
plt.show()

In [ ]:
ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)
print("훈련데이터의 정확도 : ", ridge.score(train_scaled, train_target))
print("검증데이터의 정확도 : ", ridge.score(test_scaled, test_target))

In [ ]:
print(np.sum(ridge.coef_ == 0))

In [ ]:
from sklearn.linear_model import (라쏘 규제 클래스)

lasso = Lasso()
lasso.fit(train_scaled, train_target)
print("훈련데이터의 정확도 : ", lasso.score(train_scaled, train_target))
print("검증데이터의 정확도 : ", lasso.score(test_scaled, test_target))

In [ ]:
train_score = []
test_score = []

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
for alpha in alpha_list:
  lasso = Lasso(alpha=alpha)
  lasso.fit(train_scaled, train_target)
  train_score.append(lasso.score(train_scaled, train_target))
  test_score.append(lasso.score(test_scaled, test_target))

plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel("alpha")
plt.ylabel("R^2")
plt.show()

In [ ]:
lasso = Lasso(alpha=10)
lasso.fit(train_scaled, train_target)
print("훈련데이터의 정확도 : ", lasso.score(train_scaled, train_target))
print("검증데이터의 정확도 : ", lasso.score(test_scaled, test_target))

In [ ]:
print(np.sum(lasso.coef_ == 0))
print(lasso.coef_ == 0)

In [ ]:
print(np.sum(lasso.coef_ != 0))
print(lasso.coef_)